In [1]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.external import tifffile
from skimage import transform as tf
from tqdm import tnrange
import h5py
import pandas as pd
import os

# beneath is a list of useful functions
def crop_cms(image,threshold = 85,crop_size = 300):
    '''
    get an area around the centroid of an image
    
    image (2d numpy array): original image waiting to be cropped
    threshold (float): pixels under this threshold will be included in the centroid finding algorithm
    crop_size (int) : the half size of the image
    '''
    # get center of mass
    cms = ndimage.center_of_mass(image<threshold)
    
    # set cropped image size
    xcrop = crop_size
    ycrop = crop_size
    
    # insert a dark frame if tracking fails
    if np.isnan(cms[0]) or np.isnan(cms[1]):
        return np.zeros((xcrop*2, ycrop*2)), 0, 0
    
    # convert center of mass to an integer
    xcm = int(cms[0])
    ycm = int(cms[1])

    # get cropped image boundries
    xl = xcm - xcrop
    xr = xcm + xcrop
    yl = ycm - ycrop
    yr = ycm + ycrop

    # set the correct location in the cropped image
    xst = 0
    xend = xcrop * 2
    yst = 0 
    yend = ycrop * 2

    if xl<0:
        xst = - xl
        xl = 0
    if xr > image.shape[0]:
        xend -= (xr - image.shape[0])
        xr = image.shape[0]
    if yl<0:
        yst = - yl
        yl = 0
    if yr > image.shape[1]:
        yend -= (yr - image.shape[1])
        yr = image.shape[1]

    # set the cropped image
    image_crop = np.zeros((xcrop*2, ycrop*2))
    image_crop[xst:xend,yst:yend] = image[xl:xr,yl:yr]
    return image_crop, cms[0], cms[1]

def load_avi(fname, nframes = 0, start_frame = 1):
    '''
    load an avi movie to numpy array
    
    fname (str): path to the avi file to be loaded
    nframes (int): number of frames to be loaded, if nframes is 0, will load all frames in the file
    '''
    cap = cv2.VideoCapture(fname)

    # get number of frames in the movie
    all_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # make sure number of frames is correct under default or extreme conditions
    if nframes == 0 or nframes > all_frames:
        nframes = all_frames
        
    if start_frame>= nframes:
        start_frame = 1

    #get the first frame to get the size of the frame
    ret,frame0rgb = cap.read()
    frame0 = cv2.cvtColor(frame0rgb,cv2.COLOR_BGR2GRAY)
    frame_shape = frame0.shape
    x_pix = frame_shape[0]
    y_pix = frame_shape[1]
    print('The video dimension is x = %dpx y = %dpx n = %dframes' % (x_pix,y_pix,nframes))

    #intiate numpy array
    mov = np.zeros((x_pix,y_pix,nframes-start_frame))
    mov[:,:,0] = frame0

    #load video to numpy array
    for i in tnrange(1,nframes):
        ret, frame_rgb = cap.read()
        if i >= start_frame:
            frame = cv2.cvtColor(frame_rgb,cv2.COLOR_BGR2GRAY)
            mov[:,:,i-start_frame] = frame

    print('Loading is all done! %.2f%% of the video is loaded'%100)
    return mov

def crop_avi(fname,start_frame = 0, nframes = 0,csize = 300):
    '''
    load an avi movie to numpy array
    
    fname (str): path to the avi file to be loaded
    nframes (int): number of frames to be loaded, if nframes is 0, will load all frames in the file
    '''
    
    cap = cv2.VideoCapture(fname)

    # get number of frames in the movie
    all_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if nframes == 0 or nframes > all_frames:
        nframes = all_frames
    mov_crop = np.zeros((csize*2,csize*2,nframes-start_frame))
    tr_cms = np.zeros((nframes,2))
    # make sure number of frames is correct under default or extreme conditions


    #get the first frame to get the size of the frame
    ret,frame0rgb = cap.read()
    frame0 = cv2.cvtColor(frame0rgb,cv2.COLOR_BGR2GRAY)
    frame_shape = frame0.shape
    x_pix = frame_shape[0]
    y_pix = frame_shape[1]
    print('The video dimension is x = %dpx y = %dpx n = %dframes' % (x_pix,y_pix,nframes))
    mov_crop[:,:,0],xcm,ycm = crop_cms(frame0, threshold = thrs, crop_size = csize)
    tr_cms[0,0] = xcm
    tr_cms[0,1] = ycm
    
    #load video to numpy array
    for i in tnrange(1,nframes):
        ret, frame_rgb = cap.read()
        frame = cv2.cvtColor(frame_rgb,cv2.COLOR_BGR2GRAY)
        if i >= start_frame:
            mov_crop[:,:,i-start_frame],xcm,ycm = crop_cms(frame, threshold = thrs, crop_size = csize)
            tr_cms[i-start_frame,0] = xcm
            tr_cms[i-start_frame,1] = ycm

    print('Croppinging is all done! %.2f%% of the video is loaded'%100)
    return mov_crop,tr_cms

def get_trace(fname_trail,fname_cms,start_i=0,end_i=1,scale = 1.0):
    f = h5py.File(fname_trail,'r')
    pix_size = 0.05547850208
    track_len = end_i - start_i
    dset = f['/measurement/ant_watch/buffer']
    motor_trace = dset[start_i:(start_i + track_len),:]/pix_size * scale

    cms = np.fliplr(np.load(fname_cms))[0:track_len,:]
    cms[:,1] = 1024 - cms[:,1]
    cms_mm = cms

    ant_trace = (motor_trace + cms_mm)[1:,:]
    return ant_trace

def align_frame(test_frame):
    dx = test_frame[0,0] - test_frame[3,0]
    dy = test_frame[0,1] - test_frame[3,1]
    dd = np.sqrt(dx*dx + dy*dy)
    theta = np.arccos(dy/dd)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c,-s), (s, c)))

    rotated_frame = np.matmul(R,test_frame.transpose()).transpose()
    rotated_frame -= rotated_frame[2,:]
    return rotated_frame

def get_pose(fname):
    dset = pd.read_hdf(fname)
    nframes = dset.values.shape[0]
    dset_arr = dset.values.reshape((nframes,30,3))
    dset_arr2 = dset_arr[:]
    return dset_arr2
    
def plot_pose(arr,i):
    pose_plot = plt.plot(arr[i,0:4,0],arr[i,0:4,1],
         arr[i,4:8,0],arr[i,4:8,1],
         arr[i,8:11,0],arr[i,8:11,1],
         arr[i,11:14,0],arr[i,11:14,1],
        arr[i,14:17,0],arr[i,14:17,1],
        arr[i,17:21,0],arr[i,17:21,1],
        arr[i,21:24,0],arr[i,21:24,1],
       arr[i,24:27,0],arr[i,24:27,1],
        arr[i,27:30,0],arr[i,27:30,1])
    return pose_plot

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
'''
Set parameters
'''
data_folder = '.'
sample_folder = './ant_dots/'
zoomed_view_name = 'track_mov190615-0000.avi'

# choose crop size
c_size = 200

#set the start frame
sframe = 0

#threshold for ant darkness
thrs = 85

N = 0
'''
Start transforming
'''
# load raw movie
print('loading tracking view')
mov_crop,tr_cms = crop_avi(sample_folder + zoomed_view_name, start_frame = sframe, nframes = N, csize = c_size)

# start to crop the movie
# print('Generating zoomed view...')
# mov_crop = np.zeros((csize*2,csize*2,nframes))
# for i in range(nframes):
#     mov_crop[:,:,i] = crop_cms(mov[:,:,i], threshold = thrs, crop_size = csize)
#     if i % 60 == 0:
#         print('Cropping in progress %.2f%% of the video is cropped'%(i*100.0/nframes),end = '\r')

print('Cropping is all done! %.2f%% of the video is cropped'%100)

# save the cropeed zoomed view
print('Saving zoomed view...')
mov_crop = mov_crop.transpose((2,0,1))
tifffile.imsave(sample_folder + 'zoomed_view.tif', mov_crop.astype(np.uint8))
np.save(sample_folder + 'tr_cms.npy',tr_cms)
print('Saving complete.')


loading tracking view
The video dimension is x = 1024px y = 1024px n = 1252frames


HBox(children=(IntProgress(value=0, max=1251), HTML(value='')))

C:\ProgramData\Anaconda3\lib\site-packages\scipy\ndimage\measurements.py:1301: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]



Croppinging is all done! 100.00% of the video is loaded
Cropping is all done! 100.00% of the video is cropped
Saving zoomed view...
Saving complete.
